# RxAvans lecture 4

Ok, roll up your sleeves! Today we'll write the first real functionality of our library!

We'll implement `log`, `tap`, `filter`, and `map` today.

To do this we simply need to combine the concepts of the two previous RxAvans exercise sheets: we need to combine lazy evaluation and function decorators.
The most difficult part is to understand the basic idea, so let me give you a simple example first.

In [ ]:
let doNothing previous =
    fun next ->
        previous (fun data -> 
            next data
        )

let producer observer = observer 3
let intermediate = doNothing producer

intermediate (printfn "received: %i")

received: 3


What...?! 

Don't panic!

Let's examine this example together. 

Remember we are trying to build a reactive pipeline like RxJS:

```javascript
// RxJS:
from([1,2,3])
    .pipe(
        filter(val => val % 2 == 0),
        map(val => 3 * val)
    )
    .subscribe(console.log)
```

You should be able to recognize the `producer` from the previous sheet about lazy evaluation. The producer only starts producing values when it gets called with the `observer`, so only when there is an actualy observer the reactive pipeline starts working.

Then `doNothing` also looks a lot like a lazy evaluated function... because it is! However, `producer` is at the start of the reactive pipeline, while `doNothing` is a link in the middle. So comparing it to the RxJS example `producer` is `from([1,2,3])`, while `doNothing` would be a function in `pipe(...)`, like `filter` and `map` in the example. `subscribe(console.log)` in RxJS plays the same role as `intermediate (printfn "received: %i")` in our example.

Let's try to understand how `doNothing` works. It gets called with the previous link in the chain (in this case the function `producer`). `doNothing` then returns a function that you can call with the next link in the chain (in this case the function `printfn "received: %i"`). When it gets called with the printfn it in turn calls the previous link in the chain (`producer`) with some lambda function (`fun data -> next data`). All this lambda function does is pass on the data it receives from the previous link to the next link in the chain. So we made a reactive pipeline that does nothing (hence the name `doNothing`!). 

Phew... that's a lot. Read that paragraph a couple of times and then let's make the pipeline do something useful!

Okay, first goal is to implement `log`. This should be a step in the chain that simply prints out the value that it passes along the chain. 

This actually sounds very similar to `doNothing`... All we need to do extra is add a line with `printfn` somewhere in `doNothing`. I've copied the `doNothing` below, changed the name, and added one line. Now it's a logging link!

In [ ]:
let log previous =
    fun next ->
        previous (fun data -> 
            printfn "logging: %i" data // this line was added
            next data
        )

let producer observer = observer 3
let intermediate = log producer

intermediate (printfn "received: %i")

logging: 3
received: 3


Coooool! Now you try `tap`! 

`tap` is very similar to log, but you can give it a function that gets executed instead of a hardcoded `printfn`.

I got you started with the declaration of `tap`:

In [ ]:
let tap func previous = 
    fun next ->
        previous (fun data ->
            func data
            next data
        )

let producer observer = observer 3
let print = printfn "logging through tap: %i"
let intermediate = tap print producer

intermediate (printfn "received: %i")

logging through tap: 3
received: 3


Okay, nice! Now you're ready to write a link that actually changes the pipeline and not just lets a function eavesdrop (`tap` is called that because you tap in a function).

Let's implement `map`. It takes a function and the previous links as argument, just like `tap`. But now the function maps values. The `map` should call the mapper function on a value before calling next with the mapped value. 

Try it!

In [ ]:
let map mapper previous =
    fun next ->
        previous (fun data ->
            next (mapper data)
        )

let producer observer = 
    observer 1
    observer 2
    observer 3

let intermediate = map (fun x -> 2*x) producer

intermediate (printfn "received: %i")

received: 2
received: 4
received: 6


Okay, last step for today: `filter`!

I think you already know what it should do. It also gets a function as first argument. If the function returns true the value should be passed to `next`, otherwise nothing should happen.

Go for it!

In [ ]:
let filter predicate previous =
    fun next -> 
        previous (fun data -> 
            if predicate data then next data
        )

let producer observer = 
    observer 1
    observer 2
    observer 3

let intermediate = filter (fun x -> x % 2 = 1) producer

intermediate (printfn "received: %i")

received: 1
received: 3


Nice!

We actually made a working tool!

Let's put it all together. Write a pipeline below with the following steps:

1. a map that doubles the number
2. a tap that prints "look at me being cool: \<value\>" for each value that passes through the pipeline
3. a filter that filters out all numbers that are divisible by 3

Finally subscribe to it with `printfn "received: %i"`

In [ ]:
let producer observer = for i in 1..10 do observer i 

let doubled = map (fun x -> 2*x) producer
let tapped = tap (printfn "look at me being cool: %i") doubled
let filtered = filter (fun x -> x % 3 = 0) tapped

filtered (printfn "received: %i")

look at me being cool: 2
look at me being cool: 4
look at me being cool: 6
received: 6
look at me being cool: 8
look at me being cool: 10
look at me being cool: 12
received: 12
look at me being cool: 14
look at me being cool: 16
look at me being cool: 18
received: 18
look at me being cool: 20


<span style="font-size:4em">🤩</span>